In [117]:
import wptools
from SPARQLWrapper import SPARQLWrapper2, SPARQLWrapper, JSON
import unicodedata
import pandas as pd
import numpy as np
import json
import requests
import time

In [2]:
def get_insee(commune):
    """
    returns the insee code of the commune
    """
    commune_ = unicodedata.normalize('NFD', commune.lower()).encode('ascii', 'ignore').decode("utf-8")
    url = 'https://geo.api.gouv.fr/communes?nom={c}&fields=nom,code&format=json&geometry=centre'\
              .format(c=commune_)
    exists = len(json.loads(requests.get(url).text))
    if exists>0 :
        codes = json.loads(requests.get(url).text)
        result = [code["code"] for code in codes \
                if  unicodedata.normalize('NFD', code["nom"].lower()).encode('ascii', 'ignore').decode("utf-8") == commune_]
        if len(result)>0 :
            return result
        
        return -1
    return -1

In [11]:
def dbPedia_get_wplabel(insee):
    d = SPARQLWrapper2("http://fr.dbpedia.org/sparql")

    d.setQuery(
    """
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>

    SELECT distinct ?label
    WHERE
    {
      ?ville rdfs:label ?label.
      ?ville dbo:country  <http://fr.dbpedia.org/resource/France>.
      ?ville <http://fr.dbpedia.org/property/nom> ?insee.
      filter(regex(?insee,\"^""" +insee+ """\", "i") && lang(?label) = 'fr')
      
    }
    """
    ) 
    communes = set()
    res = d.query().bindings
    for result in res:
        communes.add(result["label"].value)
        
    return communes

#### Getting the wikipedia label for each commune

In [12]:
# labels = {}
# for commune in communes:
#     insee = get_insee(commune)
#     if insee!=-1:
#         result = set()
#         for insee_ in insee:
#             label = dbPedia_get_wplabel(insee_)
#             result = result.union(label)
        
#         if(len(result)>0):
#             labels[commune] = list(result)
#         print(commune, result)
#         print("________________________________________________")
        
# with open("Data/wk_labels2.json", "w") as fp:
#     json.dump(labels,fp)

#from file
json_file = open("Data/wk_labels2.json")
labels = json.load(json_file)

### for each commune label, get the infobox of wikipedia page and extract infobox["gentilé"] 

In [87]:
gentiles = {}
for label in labels :
    wks = labels[label]
    result = set()
    for wk in wks :
        try:
            page = wptools.page(wk, lang='fr')
            parse = page.get_parse()
            infobox = parse.data["infobox"]
            if infobox is not None and "gentilé" in infobox:
                g = infobox["gentilé"]
                result.add(g)
        except:
            print("page does not exist")
            
        if len(result)>0:
            gentiles[label] = list(result)
        print(wks, result)
        
with open("Data/infobox_extracted_demonyms___.json", "w") as fp:
    json.dump(gentiles,fp)

In [64]:
page = wptools.page(wikibase='Q21979655', lang='fr').get()
page.data["infobox"]

www.wikidata.org (wikidata) Q21979655
www.wikidata.org (labels) Q1007427|P910|P242|P17|Q1007523|P1082|Q...
fr.wikipedia.org (query) Val_de_Virvée
fr.wikipedia.org (parse) 9646962
fr.wikipedia.org (restbase) /page/summary/Val de Virvée
fr.wikipedia.org (imageinfo) File:Aubie-et-Espessas Mairie.JPG
Val de Virvée (fr) data
{
  WARNINGS: <dict(1)> extracts
  assessments: <dict(1)> Communes de France
  claims: <dict(25)> P17, P31, P571, P374, P2046, P1365, P625, P13...
  description: commune française du département de la Gironde
  exhtml: <str(395)> <p><b>Val de Virvée</b> est, depuis le <time ...
  exrest: <str(294)> Val de Virvée est, depuis le 1er janvier 2016...
  extext: <str(313)> **Val de Virvée** est, depuis le 1er janvier ...
  extract: <str(1641)> <p class="mw-empty-elt"></p><p><b>Val de Vi...
  image: <list(6)> {'file': 'File:Aubie-et-Espessas Mairie.JPG', '...
  infobox: <dict(25)> nom, image, légende, région, département, ar...
  iwlinks: <list(1)> https://fr.wiktionary.org/wi

{'nom': 'Val de Virvée',
 'image': 'Aubie-et-Espessas Mairie.JPG',
 'légende': 'La mairie.',
 'région': '[[Nouvelle-Aquitaine]]',
 'département': '[[Gironde (département)|Gironde]]',
 'arrondissement': '[[Arrondissement de Blaye|Blaye]]',
 'canton': '[[Canton du Nord-Gironde]]',
 'circonscription législative': '[[Onzième circonscription de la Gironde|Onzième circonscription]]',
 'insee': '33018',
 'cp': '33240',
 'maire': 'Christophe Martial',
 'mandat maire': '[[Élections municipales de 2020 en Gironde|2020]]-2026',
 'intercomm': '[[Communauté de communes du Grand Cubzaguais]]',
 'latitude': '45.02',
 'longitude': '-0.405555555556',
 'alt mini': '13',
 'alt maxi': '64',
 'superficie': '20.77',
 'type': 'Commune rurale',
 'unité urbaine': '[[Unité urbaine de Bordeaux|Bordeaux]] <br><small>([[banlieue]])</small>',
 "aire d'attraction": "[[Aire d'attraction de Bordeaux|Bordeaux]] <br><small>(commune de la couronne)</small>",
 'population': '{{Population de France/dernière_pop}}',
 'année

In [151]:
def wikidata_get_wplabel(name):
    d = SPARQLWrapper("https://query.wikidata.org/sparql", agent="hubeauBot/1.0 (https://github.com/BRGM/hubeau)")
    d.setQuery(
    """
    SELECT distinct ?ville
    WHERE
    { 
      ?ville rdfs:label ?label.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "fr" }.
      ?ville wdt:P17 wd:Q142.
      ?ville wdt:P1448 ?insee.     
      filter(regex(?insee,"^"""+name+"""$", "i")).      
      
    }
    """
    ) 
    communes = set()
    d.setReturnFormat(JSON)
    try:
        res = d.query().convert()
        print(res)
        for result in res["results"]["bindings"]:
            communes.add(result["ville"]["value"].split("/")[-1])
        return list(communes)
    
    except KeyboardInterrupt:
        print("interrupted")
        res = d.query().convert()
        print(res)
        for result in res["results"]["bindings"]:
            communes.add(result["ville"]["value"].split("/")[-1])
        return list(communes)
    
    except:
        print("error")
        time.sleep(120)
        res = d.query().convert()
        print(res)
        for result in res["results"]["bindings"]:
            communes.add(result["ville"]["value"].split("/")[-1])
        return list(communes)
                
    

In [152]:
wikidata_get_wplabel("Aube")

{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q3194'}}, {'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q22126'}}, {'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1047537'}}]}}


['Q1047537', 'Q3194', 'Q22126']

In [196]:
df=pd.read_csv("Data/commune2021.csv")
communes_ = np.array(df["LIBELLE"])
communes, counts = np.unique(communes_, return_counts=True)
exists = json.load(open("Data/wikidata_ids1.json"))
exists2 = json.load(open("Data/wikidata_ids2.json"))
exists3 = json.load(open("Data/wikidata_ids3.json"))
exists4 = json.load(open("Data/wikidata_ids4.json"))
exists5 = json.load(open("Data/wikidata_ids5.json"))
exists6 = json.load(open("Data/wikidata_ids6.json"))
exists7 = json.load(open("Data/wikidata_ids7.json"))
exists8 = json.load(open("Data/wikidata_ids8.json"))
exists9 = json.load(open("Data/wikidata_ids9.json"))
exists10 = json.load(open("Data/wikidata_ids10.json"))
exists11 = json.load(open("Data/wikidata_ids11.json"))
exists12 = json.load(open("Data/wikidata_ids12.json"))
exists13 = json.load(open("Data/wikidata_ids13.json"))
exists14 = json.load(open("Data/wikidata_ids14.json"))
exists15 = json.load(open("Data/wikidata_ids15.json"))
exists16 = json.load(open("Data/wikidata_ids16.json"))

In [ ]:
def cond(commune):
    if commune not in exists and commune not in exists2 and commune not in exists3 and commune not in exists4 \
    and commune not in exists5 and commune not in exists6 and commune not in exists7 and commune not in exists8 \
    and commune not in exists9 and commune not in exists10 and commune not in exists11 and commune not in exists12\
    and commune not in exists13 and commune not in exists14 and commune not in exists15 and commune not in exists16:
        return True
    return False


In [ ]:
wikidata_ids = {}
k = 0
for commune in communes:
    if cond(commune):
        ids = wikidata_get_wplabel(commune)      
        if(len(ids)>0):
            wikidata_ids[commune] = ids
        print(commune, ids)
        print("________________________________________________")
        
with open("Data/wikidata_ids.json", "w") as fp:
    json.dump(wikidata_ids,fp)    

{'head': {'vars': ['ville']}, 'results': {'bindings': []}}
Allemagne-en-Provence []
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': []}}
Bors (Canton de Charente-Sud) []
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': []}}
Bors (Canton de Tude-et-Lavalette) []
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': []}}
Castillon (Canton d'Arthez-de-Béarn) []
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': []}}
Castillon (Canton de Lembeye) []
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': []}}
Château-Chinon (Campagne) []
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': []}}
Château-Chinon (Ville) []
________________________________________________
{'head': {'vars': ['

{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q835532'}}]}}
Pointis-de-Rivière ['Q835532']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1227000'}}]}}
Pointre ['Q1227000']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1141787'}}]}}
Poinville ['Q1141787']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q566847'}}]}}
Poinçon-lès-Larrey ['Q566847']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1420063'}}]}}
Poiroux ['Q1420

{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1388871'}}]}}
Pollionnay ['Q1388871']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q764074'}}]}}
Polminhac ['Q764074']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1240284'}}]}}
Polveroso ['Q1240284']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1224244'}}]}}
Pomacle ['Q1224244']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1004224'}}]}}
Pomarez ['Q1004224']
_________

{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q227149'}}]}}
Pompogne ['Q227149']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q909395'}}]}}
Pomponne ['Q909395']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1077134'}}]}}
Pomport ['Q1077134']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q174803'}}]}}
Pomps ['Q174803']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1007367'}}]}}
Pompéjac ['Q1007367']
___________________

{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q686000'}}]}}
Pont-Saint-Martin ['Q686000']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q632245'}}]}}
Pont-Saint-Pierre ['Q632245']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1049800'}}]}}
Pont-Saint-Vincent ['Q1049800']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1366500'}}]}}
Pont-Sainte-Marie ['Q1366500']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1336775'}}]}

...

{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1156222'}}]}}
Saint-Mars-d'Égrenne ['Q1156222']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1065403'}}]}}
Saint-Mars-de-Coutais ['Q1065403']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1233947'}}]}}
Saint-Mars-de-Locquenay ['Q1233947']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q957960'}}, {'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1064610'}}]}}
Saint-Mars-du-Désert ['Q957960', 'Q1064610']
________________________________________________
{'head': {'vars': ['ville'

{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q253014'}}]}}
Saint-Martin-Cantalès ['Q253014']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1022822'}}]}}
Saint-Martin-Choquel ['Q1022822']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q971108'}}]}}
Saint-Martin-Château ['Q971108']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q661576'}}]}}
Saint-Martin-Curton ['Q661576']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q735

{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1144381'}}]}}
Saint-Martin-d'Ary ['Q1144381']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1061703'}}]}}
Saint-Martin-d'Aubigny ['Q1061703']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1061756'}}]}}
Saint-Martin-d'Audouville ['Q1061756']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1141314'}}]}}
Saint-Martin-d'Auxigny ['Q1141314']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.or

{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q679283'}}]}}
Saint-Martin-de-Fontenay ['Q679283']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q520433'}}]}}
Saint-Martin-de-Fraigneau ['Q520433']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q626949'}}]}}
Saint-Martin-de-Fressengeas ['Q626949']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q264604'}}]}}
Saint-Martin-de-Fugères ['Q264604']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidat

{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q218836'}}]}}
Saint-Martin-de-Villeréal ['Q218836']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q273692'}}]}}
Saint-Martin-de-l'Arçon ['Q273692']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q468062'}}]}}
Saint-Martin-de-la-Brasque ['Q468062']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q676474'}}]}}
Saint-Martin-de-la-Cluze ['Q676474']
________________________________________________
{'head': {'vars': ['ville']}, 'results': {'bindings': [{'ville': {'type': 'uri', 'value': 'http://www.wikidata

In [194]:
wikidata_ids

{'Pithon': ['Q848562'],
 'Pittefaux': ['Q1027439'],
 'Pizay': ['Q845872'],
 'Pizieux': ['Q1225997'],
 'Piève': ['Q252460'],
 'Piégon': ['Q476011'],
 'Piégros-la-Clastre': ['Q1009442'],
 'Piégut': ['Q1093065'],
 'Piégut-Pluviers': ['Q585111'],
 'Plabennec': ['Q218240'],
 'Placey': ['Q923484'],
 'Plachy-Buyon': ['Q586773'],
 'Placy': ['Q862573'],
 'Placy-Montaigu': ['Q636350'],
 'Placé': ['Q535663'],
 'Plagne': ['Q328407', 'Q1144344'],
 'Plagnole': ['Q1143804'],
 'Plaigne': ['Q1083232'],
 'Plailly': ['Q1337799'],
 'Plaimbois-Vennes': ['Q907023'],
 'Plaimbois-du-Miroir': ['Q906948'],
 'Plaimpied-Givaudins': ['Q1172374'],
 'Plaine': ['Q22916'],
 'Plaine-Haute': ['Q288900'],
 "Plaine-d'Argenson": ['Q44012281'],
 'Plaine-de-Walsch': ['Q22242'],
 'Plaine-et-Vallées': ['Q59488794'],
 'Plainemont': ['Q904059'],
 'Plaines-Saint-Lange': ['Q1104763'],
 'Plainfaing': ['Q903654'],
 'Plainoiseau': ['Q1227062'],
 'Plaintel': ['Q209712'],
 'Plainval': ['Q1346715'],
 'Plainville': ['Q1174267', 'Q1073265

In [195]:
with open("Data/wikidata_ids16.json", "w") as fp:
    json.dump(wikidata_ids,fp)

In [ ]:
ids = ....
gentiles = {}
for label, ids in wikidata_ids.items() :
    result = set()
    for id in ids :
        try:
            page = wptools.page(wikibase=id, lang='fr').get()
            infobox = page.data["infobox"]
            if infobox is not None and "gentilé" in infobox:
                g = infobox["gentilé"]
                result.add(g)
        except:
            print("page does not exist")
          
    if len(result)>0:
        gentiles[label] = list(result)
        
    print(label, result)
        
with open("Data/wikidata_infobox_extracted_demonyms.json", "w") as fp:
    json.dump(gentiles,fp)

### Preprocess extracted text

In [3]:
#from file
json_file = open("Data/infobox_extracted_demonyms2.json")
gentiles = json.load(json_file)
len(gentiles)

22792

In [6]:
import re

gentiles_ = {}
for com in gentiles:
    list_g = gentiles[com]
    result = set()
    for g in list_g:
        g1 = re.sub("<.*>", ",", g)
        g2 = re.sub("\sou\s|\set\s|;|/", ",", g1)
        print(g2)
        split_g = re.split("\s?,,?\s?", g2)
        print(split_g)
        for split_g_ in split_g:  
            result.add(split_g_)
    result = list(result)
    gentiles_[com] = result   
    print(com, list_g, result)

Aastais, Aastaises
['Aastais', 'Aastaises']
Aast ['Aastais, Aastaises'] ['Aastais', 'Aastaises']
Abainvillois, Abainvilloises
['Abainvillois', 'Abainvilloises']
Abainville ['Abainvillois, Abainvilloises'] ['Abainvilloises', 'Abainvillois']
Abancourtois, Abancourtoises
['Abancourtois', 'Abancourtoises']
Abancourt ['Abancourtois, Abancourtoises'] ['Abancourtois', 'Abancourtoises']
Abaucourtois, Abaucourtoises
['Abaucourtois', 'Abaucourtoises']
Abaucourt ['Abaucourtois, Abaucourtoises'] ['Abaucourtoises', 'Abaucourtois']
Abaucourtois, Abaucourtoises
['Abaucourtois', 'Abaucourtoises']
Abaucourt-Hautecourt ['Abaucourtois, Abaucourtoises'] ['Abaucourtoises', 'Abaucourtois']
Abbanais, Abbanaises
['Abbanais', 'Abbanaises']
Abbans-Dessous ['Abbanais, Abbanaises'] ['Abbanais', 'Abbanaises']
Abbanais, Abbanaises
['Abbanais', 'Abbanaises']
Abbans-Dessus ['Abbanais, Abbanaises'] ['Abbanais', 'Abbanaises']
Abbarois, Abbaroises
['Abbarois', 'Abbaroises']
Abbaretz ['Abbarois, Abbaroises'] ['Abbaroises

Arsacais
['Arsacais']
Arsac ['Arsacais'] ['Arsacais']
Arsacois(-ses)
['Arsacois(-ses)']
Arsac-en-Velay ['Arsacois(-ses)'] ['Arsacois(-ses)']
Arsaguais
['Arsaguais']
Arsague ['Arsaguais'] ['Arsaguais']
Les Arsonvallois
['Les Arsonvallois']
Arsonval ['Les Arsonvallois'] ['Les Arsonvallois']
Barochers
['Barochers']
Arsure-Arsurette ['Barochers'] ['Barochers']
Arsiens, Arsiennes
['Arsiens', 'Arsiennes']
Arsy ['Arsiens, Arsiennes'] ['Arsiennes', 'Arsiens']
Arcquois, Arcquoises
['Arcquois', 'Arcquoises']
Art-sur-Meurthe ['Arcquois, Arcquoises'] ['Arcquois', 'Arcquoises']
Artagnanais
['Artagnanais']
Artagnan ['Artagnanais'] ['Artagnanais']
Artaisiens
['Artaisiens']
Artaix ['Artaisiens'] ['Artaisiens']
Artouinois, Artouinoises
['Artouinois', 'Artouinoises']
Artalens-Souin ['Artouinois, Artouinoises'] ['Artouinoises', 'Artouinois']
Artannais, Artannaise
['Artannais', 'Artannaise']
Artannes-sur-Indre ['Artannais, Artannaise'] ['Artannaise', 'Artannais']
Artannais
['Artannais']
Artannes-sur-Thoue

Basse-Pointe ['Pointois / Pointoise'] ['Pointoise', 'Pointois']
Rentgenois
['Rentgenois']
Basse-Rentgen ['Rentgenois'] ['Rentgenois']
Basse-Terrien(ne)s
['Basse-Terrien(ne)s']
Basse-Terre ['Basse-Terrien(ne)s'] ['Basse-Terrien(ne)s']
Bassurois(es)
['Bassurois(es)']
Basse-sur-le-Rupt ['Bassurois(es)'] ['Bassurois(es)']
Bassembergeois
['Bassembergeois']
Bassemberg ['Bassembergeois'] ['Bassembergeois']
Bassenevillais
['Bassenevillais']
Basseneville ['Bassenevillais'] ['Bassenevillais']
Bassenais
['Bassenais']
Bassinots
['Bassinots']
Bassens ['Bassenais', 'Bassinots'] ['Bassinots', 'Bassenais']
Bassésiens
['Bassésiens']
Basses ['Bassésiens'] ['Bassésiens']
Basseusiens
['Basseusiens']
Basseux ['Basseusiens'] ['Basseusiens']
Bassevellois
['Bassevellois']
Bassevelle ['Bassevellois'] ['Bassevellois']
Bassignacois, Basssignacoises
['Bassignacois', 'Basssignacoises']
Bassignac ['Bassignacois, Basssignacoises'] ['Basssignacoises', 'Bassignacois']
Bassignacois
['Bassignacois']
Bassignac-le-Bas ['B

['Bortois']
Bort-les-Orgues ['Bortois'] ['Bortois']
Borvillois, Borvilloises
['Borvillois', 'Borvilloises']
Borville ['Borvillois, Borvilloises'] ['Borvillois', 'Borvilloises']
Bordelois, Bordeloises
['Bordelois', 'Bordeloises']
Bosc-Bordel ['Bordelois, Bordeloises'] ['Bordeloises', 'Bordelois']
Bosc-Berengerois, Bosc-Berengeroises
['Bosc-Berengerois', 'Bosc-Berengeroises']
Bosc-Bérenger ['Bosc-Berengerois, Bosc-Berengeroises'] ['Bosc-Berengerois', 'Bosc-Berengeroises']
Bosc-Guérardais, Bosc-Guérardaises
['Bosc-Guérardais', 'Bosc-Guérardaises']
Bosc-Guérard-Saint-Adrien ['Bosc-Guérardais, Bosc-Guérardaises'] ['Bosc-Guérardais', 'Bosc-Guérardaises']
Boshionnais, Boshionnaises
['Boshionnais', 'Boshionnaises']
Bosc-Hyons ['Boshionnais, Boshionnaises'] ['Boshionnaises', 'Boshionnais']
Bosc-Mesnillois, Bosc-Mesnilloises
['Bosc-Mesnillois', 'Bosc-Mesnilloises']
Bosc-Mesnil ['Bosc-Mesnillois, Bosc-Mesnilloises'] ['Bosc-Mesnillois', 'Bosc-Mesnilloises']
Bolhardais
['Bolhardais']
Bosc-le-Hard [

Buigny-l'Abbé ['Buignotins'] ['Buignotins']
Bistiers
['Bistiers']
Buigny-lès-Gamaches ['Bistiers'] ['Bistiers']
Buirois, Buiroises
['Buirois', 'Buiroises']
Buire-au-Bois ['Buirois, Buiroises'] ['Buiroises', 'Buirois']
Buirois
['Buirois']
Buire-sur-l'Ancre ['Buirois'] ['Buirois']
Buironfossois(es)
['Buironfossois(es)']
Buironfosse ['Buironfossois(es)'] ['Buironfossois(es)']
Buxois, Buxoises
['Buxois', 'Buxoises']
Buis-les-Baronnies ['Buxois, Buxoises'] ['Buxoises', 'Buxois']
Buissardins
['Buissardins']
Buissard ['Buissardins'] ['Buissardins']
Buissonnais, Buissonnaises
['Buissonnais', 'Buissonnaises']
Buisson ['Buissonnais, Buissonnaises'] ['Buissonnais', 'Buissonnaises']
Bulatois, Bulatoise
['Bulatois', 'Bulatoise']
Bulat-Pestivien ['Bulatois, Bulatoise'] ['Bulatoise', 'Bulatois']
Bulgnévillois(es)
['Bulgnévillois(es)']
Bulgnéville ['Bulgnévillois(es)'] ['Bulgnévillois(es)']
Bulhonois, Bulhonoises
['Bulhonois', 'Bulhonoises']
Bulhon ['Bulhonois, Bulhonoises'] ['Bulhonois', 'Bulhonoises

...

Saint-Sulpice-sur-Lèze ['Saint-Sulpiciens'] ['Saint-Sulpiciens']
Saint-Sulpiciens
['Saint-Sulpiciens']
Saint-Sulpice-sur-Risle ['Saint-Sulpiciens'] ['Saint-Sulpiciens']
Sulpiciens, Sulpiciennes
['Sulpiciens', 'Sulpiciennes']
Saint-Supplet ['Sulpiciens, Sulpiciennes'] ['Sulpiciens', 'Sulpiciennes']
Saint-Sylvanais, Saint-Sylvanaises
['Saint-Sylvanais', 'Saint-Sylvanaises']
Saint-Sylvain ['Saint-Sylvanais, Saint-Sylvanaises'] ['Saint-Sylvanais', 'Saint-Sylvanaises']
Saint-Sylvestrins
['Saint-Sylvestrins']
Saint-Sylvestrois
['Saint-Sylvestrois']
Saint-Sylvestre ['Saint-Sylvestrins', 'Saint-Sylvestrois'] ['Saint-Sylvestrins', 'Saint-Sylvestrois']
Cappelois
['Cappelois']
Saint-Sylvestre-Cappel ['Cappelois'] ['Cappelois']
Pragoulinois
['Pragoulinois']
Saint-Sylvestre-Pragoulin ['Pragoulinois'] ['Pragoulinois']
Paroupians
['Paroupians']
Symphoriennais
['Symphoriennais']
Saint-Symphorinois
['Saint-Symphorinois']
Saint-Symphorien ['Paroupians', 'Symphoriennais', 'Saint-Symphorinois'] ['Saint-Sy

Servon ['Servonnais'] ['Servonnais']
Servonis, Servonies
['Servonis', 'Servonies']
Servon-Melzicourt ['Servonis, Servonies'] ['Servonies', 'Servonis']
Servonnais
['Servonnais']
Servon-sur-Vilaine ['Servonnais'] ['Servonnais']
Chirve
['Chirve']
Servoz ['Chirve'] ['Chirve']
Gaudiauds, Gaudiaudes
['Gaudiauds', 'Gaudiaudes']
Sery ['Gaudiauds, Gaudiaudes'] ['Gaudiauds', 'Gaudiaudes']
Serprinois, Serprinoise
['Serprinois', 'Serprinoise']
Serzy-et-Prin ['Serprinois, Serprinoise'] ['Serprinois', 'Serprinoise']
Serécourtois, Serécourtoises
['Serécourtois', 'Serécourtoises']
Serécourt ['Serécourtois, Serécourtoises'] ['Serécourtoises', 'Serécourtois']
Serémangeois,Erzangeois
['Serémangeois', 'Erzangeois']
Serémange-Erzange ['Serémangeois et Erzangeois'] ['Serémangeois', 'Erzangeois']
Setquois
['Setquois']
Setques ['Setquois'] ['Setquois']
Seugissois(es),Cacoin(es)
['Seugissois(es)', 'Cacoin(es)']
Seugy ['Seugissois(es) ou Cacoin(es)'] ['Seugissois(es)', 'Cacoin(es)']
Seuillétois, Seuillétoises
[

Touryçois, Touryçoises
['Touryçois', 'Touryçoises']
Toury-Lurcy ['Touryçois, Touryçoises'] ['Touryçoises', 'Touryçois']
Toréins
['Toréins']
Toury-sur-Jour ['Toréins'] ['Toréins']
Toussaintais, Toussaintaises
['Toussaintais', 'Toussaintaises']
Toussaint ['Toussaintais, Toussaintaises'] ['Toussaintais', 'Toussaintaises']
Nobeltussois
['Nobeltussois']
Toussus-le-Noble ['Nobeltussois'] ['Nobeltussois']
Toutainvillais
['Toutainvillais']
Toutainville ['Toutainvillais'] ['Toutainvillais']
Toutencourtois
['Toutencourtois']
Toutencourt ['Toutencourtois'] ['Toutencourtois']
Toutensois
['Toutensois']
Toutens ['Toutensois'] ['Toutensois']
Toutlemondais
['Toutlemondais']
Toutlemonde ['Toutlemondais'] ['Toutlemondais']
Touvoisiens
['Touvoisiens']
Touvois ['Touvoisiens'] ['Touvoisiens']
Tolvériens
['Tolvériens']
Touvre ['Tolvériens'] ['Tolvériens']
Touvéracois
['Touvéracois']
Touvérac ['Touvéracois'] ['Touvéracois']
Touzacois
['Touzacois']
Touzac ['Touzacois'] ['Touzacois']
Touétois
['Touétois']
Touë

Vichy ['Vichyssois'] ['Vichyssois']
Vichèrois(e)
['Vichèrois(e)']
Vichères ['Vichèrois(e)'] ['Vichèrois(e)']
Vicolais
['Vicolais']
Vico ['Vicolais'] ['Vicolais']
Vicquois
['Vicquois']
Vicois, Vicoises
['Vicois', 'Vicoises']
Vicquois, Vicquoises
['Vicquois', 'Vicquoises']
Vicquelots, Vicquelottes
['Vicquelots', 'Vicquelottes']
Vicq ['Vicquois', 'Vicois, Vicoises', 'Vicquois, Vicquoises', 'Vicquelots, Vicquelottes'] ['Vicoises', 'Vicquoises', 'Vicquois', 'Vicquelottes', 'Vicquelots', 'Vicois']
Vicquois
['Vicquois']
Vicq-Exemplet ['Vicquois'] ['Vicquois']
Vicquois
['Vicquois']
Vicq-d'Auribat ['Vicquois'] ['Vicquois']
Vicquois
['Vicquois']
Vicq-sur-Breuilh ['Vicquois'] ['Vicquois']
Vicquois
['Vicquois']
Vicq-sur-Gartempe ['Vicquois'] ['Vicquois']
Vicquois
['Vicquois']
Vicq-sur-Nahon ['Vicquois'] ['Vicquois']
Vidayons
['Vidayons']
Vidai ['Vidayons'] ['Vidayons']
Vidaillats,Vidaillates
['Vidaillats', 'Vidaillates']
Vidaillat ['Vidaillats et Vidaillates'] ['Vidaillats', 'Vidaillates']
Vidauba

Épéhiens
['Épéhiens']
Épehy ['Épéhiens'] ['Épéhiens']
Épeignois
['Épeignois']
Épeigné-les-Bois ['Épeignois'] ['Épeignois']
Spinacois
['Spinacois']
Épeigné-sur-Dême ['Spinacois'] ['Spinacois']
Spinosiens
['Spinosiens']
Épenouse ['Spinosiens'] ['Spinosiens']
Spinoyens
['Spinoyens']
Épenoy ['Spinoyens'] ['Spinoyens']
Épenédois
['Épenédois']
Épenède ['Épenédois'] ['Épenédois']
Sparcirots
['Sparcirots']
Épercieux-Saint-Paul ['Sparcirots'] ['Sparcirots']
Éperlecquois
['Éperlecquois']
Éperlecques ['Éperlecquois'] ['Éperlecquois']
Sparnaciens
['Sparnaciens']
Épernay ['Sparnaciens'] ['Sparnaciens']
Sparnoniens
['Sparnoniens']
Épernon ['Sparnoniens'] ['Sparnoniens']
Épervanais
['Épervanais']
Épervans ['Épervanais'] ['Épervanais']
Épiais-Rhussiens
['Épiais-Rhussiens']
Épiais-Rhus ['Épiais-Rhussiens'] ['Épiais-Rhussiens']
Épiaisois,Épiaisiens
['Épiaisois', 'Épiaisiens']
Épiais-lès-Louvres ['Épiaisois ou Épiaisiens'] ['Épiaisiens', 'Épiaisois']
Spicarien(ne)s
['Spicarien(ne)s']
Épiedois
['Épiedois'

In [7]:
with open("Data/infobox_extracted_demonyms3.json", "w") as fp:
    json.dump(gentiles_,fp)